In [11]:
# Dependencies
import os
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

<b>Step 1 - Scraping<b/>

Nasa Mars News

In [12]:
#url of page to be scraped
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [13]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\berry\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [14]:
#Create soup
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [18]:
# Examine the soup and look for a div withe the class 'content_title'
news_titles=soup.find_all('div', class_='content_title')
# Iterate through the results to create the news_title list
news_title_list=[]
for title in news_titles:
    news_title_list.append(title.text.strip())

In [21]:
#Examine the soup and use the search results to create the news_teaser list
teaser_text_list=[]
news_teaser_results=soup.find_all('div', class_='article_teaser_body')
for i in news_teaser_results:
    teaser_text_list.append(i.text.strip())

JPL Mars Space Images

In [22]:
#Visit the second site and scrape the featured image/the most recent image
#Create soup for the site
url2 = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(url2)
html2=browser.html
soup2=BeautifulSoup(html2,'html.parser')

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" lang="en"><!--<![endif]--><head>
<title>Image Gallery</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/><meta content="#ba0c2f" data-n-head="ssr" name="msapplication - TileColor"/><meta content="/ms-icon-144x144.png" data-n-head="ssr" name="msapplication - TileImage"/><meta content="#ba0c2f" data-n-head="ssr" name="theme - color"/><meta content="IE=Edge" data-n-head="ssr" http-equiv="X-UA-Compatible"/><meta content="https://jpl.nasa.gov/" data-hid="author" data-n-head="ssr" name="author"/><meta content="https://jpl.nasa.gov/" data-hid="publisher" data-n-head="ssr" name="publisher"/><meta content="JPL" data-hid="apple-mobile-web-app-title" data-n-head="ssr" name="apple-mobile-web-app-title"/><meta content="#03112b" data-hid="theme-color" data-n-head="ssr" name="theme-color"/><meta content="website" data-hid="og:type" data-n-head="ssr" name="o

In [24]:
#Scrape the image and save the url
featured_image_url=soup2.find_all("img", class_="BaseImage")[0]["src"]

Mars Facts Table

In [25]:
#Visit the thrid site
url3='https://space-facts.com/mars/'

In [27]:
#Create a soup, extract the table, and save the results as a string
browser.visit(url3)
html3=browser.html
soup3=BeautifulSoup(html3,"html.parser")
soup3_results=soup3.find("table","tablepress tablepress-id-p-mars")
soup3_html_content=str(soup3_results)

Mars Hemispheres

In [31]:
#Visit the site, make a soup, click on the links, visit the secondary site, extract the full url, return to primary site,
#Repeat process

#Vist site
url4='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)
html4=browser.html
#make a soup
soup4=BeautifulSoup(html4,"html.parser")
#search soup for image
soup4_results=soup4.find_all("div", "description")

#create empty containers
hemi_name_list=[]
href_list=[]
furl_list=[]
image_object_dict={}
image_object_list=[]

#iterate through soup results and create a list of dictionaries
for i in soup4_results:
    hemi_name_list.append(i.a.text)
    #print(hemi_name_list)
    href_list.append(i.a['href'])
    #print(href_list)
    browser.links.find_by_partial_text("Enhanced").click()
    time.sleep(1)
    htmli=browser.html
    #print(html5)
    soupi=BeautifulSoup(htmli,"html.parser")
    soupi_results=soupi.find("a",text="Sample")
    #print(soupi_results)
    furl_list.append(soupi_results['href'])
    #print(furl_list)
    image_object_dict["title"]=i.a.text
    image_object_dict["img_url"]=soupi_results['href']
    image_object_list.append(image_object_dict.copy())

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [34]:
#save scraped data as a dict
mars_scraped_data={"Headline":news_title_list,"Teaser":teaser_text_list,"JPLFeaturedImage":featured_image_url,"MarsTable":soup3_html_content,"MaritanHemispherePhotos":image_object_list}
#close the browser
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x000001D446C61550>>